In [1]:
import torch
import torchvision.models as models
import os
import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from itertools import groupby
from json import dumps, loads
from nltk.tokenize import word_tokenize
from os import remove
from os.path import exists
from PIL import Image, UnidentifiedImageError
from random import sample
from re import sub
from queue import Queue
from threading import Lock, Thread
from torch.nn import DataParallel, Sequential
from zipfile import ZipFile

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class ImagePreprocessor():
    def __init__(self, tweet, tag, cnn, image_dict, transform, *args,
                 **kwargs):
        # self.__queue = queue
        # self.__bucket = bucket
        self.tweet = tweet
        self.tag = tag
        self.__cnn = cnn
        self.__image_dict = image_dict
        self.__transform = transform

#         super().__init__(*args, **kwargs)

    def __l2norm(self, X):
        return torch.div(X, torch.pow(X, 2).sum(dim=1, keepdim=True).sqrt())

    def run(self):
        id_str = self.tweet[1]

        filename = str(self.tweet[0])+"-0.jpg"
        key = "/content/drive/MyDrive/인스타/hashtag/"+tag+"/"+filename

        try:
            image = Image.open(key).convert("RGB")
            image = self.__transform(image).unsqueeze(0)

            if torch.cuda.is_available():
                image = image.cuda()

            features = self.__l2norm(self.__cnn(image))
            features = features.clone().detach().cpu()
            self.__image_dict[id_str] = features
        except:
            pass

        if exists(filename):
            remove(filename)

In [ ]:
def precompute_images(filename, tag, transform):
    cnn = DataParallel(models.__dict__["resnet152"](pretrained=True))

    if torch.cuda.is_available():
        cnn = cnn.cuda()

    cnn.module.fc = Sequential()
    df = pd.read_csv(filename, encoding='utf-8-sig')

    # for i in range(len(df)):
    #   img_path = os.path.join(path, tag , str(i)+'-0.jpg')
    #   try:
    #       input_img = Image.open(img_path)
    #   except:
    #       continue

    image_dict, fn = {}, filename[:filename.rfind(".")] + "_img_feat.pt" # 0-0_img_feat.pt
    for i in tqdm.tqdm(range(len(df))):
        ImagePreprocessor(df.iloc[i], tag, cnn, image_dict, transform).run()
    print(image_dict)
    torch.save(image_dict, fn)


In [ ]:
def get_transform():

    t_list = [transforms.RandomResizedCrop(224),
               transforms.RandomHorizontalFlip()]


    normalizer = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225])
    return transforms.Compose(t_list + [transforms.ToTensor(), normalizer])

In [ ]:
path = "/content/drive/MyDrive/인스타/hashtag"
hashtags = os.listdir(path)
print(hashtags)
for tag in hashtags:
    if(tag[0] != '.'):
        output_path = os.path.join(path, tag, tag + '.csv')
        check_path = os.path.join(path, tag, tag + '_img_feat.pt')
        if os.path.isfile(check_path):
            continue
        else :
            print(output_path)
            transform = get_transform()
            precompute_images(output_path,tag,transform)
            # for i in range(len(df)):
            #   img_path = os.path.join(path, tag , str(i)+'-0.jpg')
            #   try:
            #       input_img = Image.open(img_path)
            #       precompute_images(img_path, str(i), a)

            #   except:
            #       continue
            # break

['love', 'nofilter', 'photography', 'happy', 'smile', 'model', 'summer', 'cute', 'beauty', 'nature', 'dog', 'instagram', 'music', 'beach', 'sunset', 'style', 'life', 'instadaily', 'photo', 'food', 'fitness', 'family', 'motivation', 'friends', 'amazing', 'makeup', 'foodporn', 'me', 'instamood', 'ootd', 'picoftheday', 'design', 'hair', 'flowers', 'cat', 'vscocam', 'sky', 'bestoftheday', 'vsco', 'artist', 'drawing', 'girl', 'dogsofinstagram', 'handmade', 'instafood', 'photographer', 'wedding', 'party', 'girls', 'healthy', 'lol', 'swag', 'cool', 'foodie', 'blackandwhite', 'lifestyle', 'london', 'black', 'home', 'christmas', 'instapic', 'memes', 'holiday', 'sea', 'blue', 'pink', '.ipynb_checkpoints']
/content/drive/MyDrive/인스타/hashtag/black/black.csv


100%|██████████| 384/384 [04:50<00:00,  1.32it/s]


{'lauraloureiro252': tensor([[0.0361, 0.0230, 0.0319,  ..., 0.0207, 0.0184, 0.0210]]), 'arian_aguilarr': tensor([[0.0344, 0.0206, 0.0352,  ..., 0.0201, 0.0204, 0.0214]]), 'earn_with_selena': tensor([[0.0343, 0.0222, 0.0354,  ..., 0.0190, 0.0164, 0.0170]]), 'frican_banies': tensor([[0.0395, 0.0232, 0.0385,  ..., 0.0179, 0.0158, 0.0200]]), 'islam_mannsour': tensor([[0.0341, 0.0230, 0.0396,  ..., 0.0181, 0.0160, 0.0213]]), 'persianmonochrome': tensor([[0.0405, 0.0215, 0.0412,  ..., 0.0203, 0.0159, 0.0198]]), 'justblackeats': tensor([[0.0366, 0.0240, 0.0331,  ..., 0.0184, 0.0139, 0.0195]]), 'welenygirl': tensor([[0.0352, 0.0230, 0.0434,  ..., 0.0174, 0.0181, 0.0193]]), 'singharun48': tensor([[0.0392, 0.0201, 0.0417,  ..., 0.0157, 0.0212, 0.0186]]), 'pi.ntu9918': tensor([[0.0359, 0.0214, 0.0321,  ..., 0.0174, 0.0155, 0.0200]]), 'black_model_jitu': tensor([[0.0353, 0.0225, 0.0372,  ..., 0.0180, 0.0192, 0.0196]]), 'obsidiana__ind': tensor([[0.0365, 0.0220, 0.0385,  ..., 0.0190, 0.0170, 0.0177

100%|██████████| 355/355 [04:12<00:00,  1.40it/s]


{'sheldonhicksphotography': tensor([[0.0360, 0.0214, 0.0367,  ..., 0.0185, 0.0163, 0.0182]]), 'markus.rau.31': tensor([[0.0373, 0.0228, 0.0410,  ..., 0.0209, 0.0169, 0.0214]]), 'alexandrakfarmer': tensor([[0.0370, 0.0215, 0.0414,  ..., 0.0205, 0.0165, 0.0200]]), 'scottdeputyrealtor': tensor([[0.0404, 0.0216, 0.0347,  ..., 0.0183, 0.0180, 0.0197]]), 'cardscustoms': tensor([[0.0396, 0.0262, 0.0360,  ..., 0.0177, 0.0155, 0.0157]]), 'ironstone_furnitureandfire': tensor([[0.0397, 0.0230, 0.0326,  ..., 0.0186, 0.0183, 0.0181]]), 'thaoheartkitchen': tensor([[0.0373, 0.0250, 0.0370,  ..., 0.0202, 0.0173, 0.0188]]), 'southcalifoodie': tensor([[0.0363, 0.0217, 0.0364,  ..., 0.0172, 0.0163, 0.0185]]), 'interiavel': tensor([[0.0372, 0.0211, 0.0375,  ..., 0.0196, 0.0160, 0.0185]]), 'aestheticdno': tensor([[0.0341, 0.0214, 0.0383,  ..., 0.0183, 0.0154, 0.0201]]), 'goldenivyrose': tensor([[0.0384, 0.0204, 0.0396,  ..., 0.0195, 0.0196, 0.0205]]), 'annalohsfrealtor': tensor([[0.0398, 0.0228, 0.0430,  .

...

{'creatpix': tensor([[0.0448, 0.0226, 0.0361,  ..., 0.0187, 0.0184, 0.0178]]), 'bunly_lee': tensor([[0.0390, 0.0215, 0.0396,  ..., 0.0178, 0.0168, 0.0199]]), 'collants_queenz': tensor([[0.0387, 0.0219, 0.0365,  ..., 0.0183, 0.0179, 0.0197]]), 'photomagic_d.j': tensor([[0.0414, 0.0207, 0.0437,  ..., 0.0201, 0.0149, 0.0194]]), 'julia173333': tensor([[0.0374, 0.0239, 0.0389,  ..., 0.0184, 0.0195, 0.0203]]), 'mae.yyy000': tensor([[0.0366, 0.0225, 0.0393,  ..., 0.0185, 0.0188, 0.0204]]), 'ulku_kurt22': tensor([[0.0393, 0.0225, 0.0406,  ..., 0.0199, 0.0160, 0.0201]]), 'cdnzgf2': tensor([[0.0309, 0.0211, 0.0396,  ..., 0.0184, 0.0154, 0.0206]]), 'changethesea': tensor([[0.0374, 0.0208, 0.0395,  ..., 0.0175, 0.0136, 0.0212]]), 'elalexolivares': tensor([[0.0368, 0.0204, 0.0376,  ..., 0.0167, 0.0190, 0.0192]]), 'pankajsuiwall': tensor([[0.0350, 0.0239, 0.0342,  ..., 0.0175, 0.0170, 0.0147]]), 'zeb9drf8': tensor([[0.0319, 0.0206, 0.0322,  ..., 0.0189, 0.0162, 0.0183]]), 'engrafft': tensor([[0.0381

100%|██████████| 321/321 [04:03<00:00,  1.32it/s]


{'nakedhawaii': tensor([[0.0441, 0.0218, 0.0416,  ..., 0.0177, 0.0160, 0.0212]]), 'mitespaidecuina': tensor([[0.0383, 0.0240, 0.0410,  ..., 0.0199, 0.0172, 0.0179]]), 'marcellakasparart': tensor([[0.0403, 0.0222, 0.0409,  ..., 0.0189, 0.0181, 0.0193]]), 'piwirock': tensor([[0.0351, 0.0241, 0.0341,  ..., 0.0183, 0.0167, 0.0221]]), 'aksfisher': tensor([[0.0363, 0.0231, 0.0398,  ..., 0.0198, 0.0160, 0.0180]]), 'ellen.ephoto': tensor([[0.0361, 0.0239, 0.0388,  ..., 0.0205, 0.0169, 0.0201]]), 'mcjoyfulart': tensor([[0.0338, 0.0187, 0.0386,  ..., 0.0182, 0.0197, 0.0197]]), 'ask_pochi_and_usagi': tensor([[0.0376, 0.0221, 0.0380,  ..., 0.0217, 0.0157, 0.0172]]), 'silver.nitrates': tensor([[0.0395, 0.0208, 0.0435,  ..., 0.0221, 0.0159, 0.0159]]), 'onstreetpr': tensor([[0.0348, 0.0203, 0.0359,  ..., 0.0167, 0.0182, 0.0184]]), 'elvisthebluesharpei': tensor([[0.0383, 0.0229, 0.0398,  ..., 0.0183, 0.0187, 0.0168]]), 'farci_davide91': tensor([[0.0383, 0.0251, 0.0394,  ..., 0.0210, 0.0168, 0.0221]]),

100%|██████████| 401/401 [04:47<00:00,  1.40it/s]

{'kitschblade_vintage': tensor([[0.0370, 0.0235, 0.0390,  ..., 0.0178, 0.0179, 0.0188]]), 'iceboxxcustoms': tensor([[0.0388, 0.0231, 0.0372,  ..., 0.0183, 0.0158, 0.0184]]), 'skyyyydaddy': tensor([[0.0386, 0.0206, 0.0339,  ..., 0.0206, 0.0170, 0.0211]]), 'twistonline': tensor([[0.0386, 0.0209, 0.0362,  ..., 0.0179, 0.0176, 0.0201]]), 'buitenparadijsje': tensor([[0.0386, 0.0247, 0.0397,  ..., 0.0193, 0.0165, 0.0190]]), 'shamitdasgupta': tensor([[0.0378, 0.0220, 0.0379,  ..., 0.0180, 0.0172, 0.0184]]), 'ninotbh': tensor([[0.0360, 0.0239, 0.0405,  ..., 0.0212, 0.0160, 0.0172]]), 'shannanlifecoach': tensor([[0.0368, 0.0240, 0.0366,  ..., 0.0181, 0.0191, 0.0211]]), 'stephaniee.lar': tensor([[0.0328, 0.0223, 0.0384,  ..., 0.0201, 0.0169, 0.0232]]), 'ahyoung_makeup': tensor([[0.0398, 0.0220, 0.0410,  ..., 0.0191, 0.0189, 0.0199]]), 'espaco_eternizze_emilly_kauane': tensor([[0.0388, 0.0236, 0.0407,  ..., 0.0185, 0.0173, 0.0158]]), 'tamaraahdz': tensor([[0.0350, 0.0251, 0.0376,  ..., 0.0190, 0.

In [ ]:
df2 = pd.read_csv('./hashtag/party/party.csv')

df2.iloc[1][0]
# queue2
# "/".join(["images", "filename"])

In [104]:
df = pd.read_csv('/content/drive/MyDrive/인스타/hashtag/amazing/amazing.csv')

print(df.iloc[0][4])

#index = str.find('b')

total_photo2018📸@barbi.giangiu 🆒💯😍🇮🇹......#roma #castelsantangelo #amazing #rome #skyb#Photo #photographie #photography #photogram #instagram #instalike #instafashion #cool #italy #italia #fff #flf #likeforlike #love #nature #likeforlikeback #liketime #likes #likeme #world #instanature #4like4like #4like55분


# #_제거

In [114]:
post_text = "" 
for i in range(len(df)):
  post_text= ''
  description = df.iloc[i][4]
  idx = description.find('#')
  description = description[:idx]
  print(description)
  df['text'][i] = description

total_photo2018📸@barbi.giangiu 🆒💯😍🇮🇹......
elianndelvalleEn esta foto me veo gordita pero aja, a mi me gusto y ya.🤭..Que hacen ?❤11시
pamxtinafighterCredit : @xtinaquebec 🥰@xtina 
_cateblanchettfanpage_She gave me her usual smile, which stopped my breath and heart.  I couldn't imagine a more splendid angel.❤️❤️
anaalokeh💙💜💙
akrimna_fishcenter
robertomullanuLight Swords......... 
pandora.carmainSabtu malam minggu enaknya wisata kuliner aja 😋
piero_hb☔️🌧••••••••
chintu_official55
cricket4fun@josbuttler against Bambai be like.......© RESEBH PENT 2.0....
mindfullness001
chia_lc
chessturFever Dream Pt. 5 : Love Differently....
andreasschnitzlhuberSpringtime ......
insta_klub_44
vishal779755
kotaflorist❤ 👰❤
sarahsaleh91🦊27
jadorbucProve outfit per il @decadence_official Direttamente dalla mia cucina 😅.....
daniele.zinna🔥🐲1시
fairyheartaqforyou 
javad_.golzari
noeeangeliist🍁rt
alloccabimboBOW SANDALS WHITE🥛Vi aspettiamo in via Madonna dell'Arco 151,Sant'Anastasia (NA)📍
jewelplanethydAll New Dia

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


_irtarun_🌸🌿🌺I love the way you are🤙🌿...............😍😍😍😍😍..
questdentalcareDreaming of that dazzling white smile! 🌜💭Here at Quest Dental Care we offer two amazing teeth whitening treatment options:🏡 Enlighten Home Teeth Whitening💺 Enlighten Guaranteed Whiter Shade Teeth WhiteningBoth treatments help our patients achieve amazing results either with our dental team or from the comfort of their home. Allowing all our patients to fit treatment around their busy personal life. 😁🙌Still unsure which one is best? Chat with our friendly team today!📞 01473 597527🌐 www.questdentalipswich.com10시
prachi_arora_prayagraj_foodieYum yum 😋 
mixej14425
sandra_bergx3Overthinking will destroy your mood. Breath and let go 🕊 ___________________________________
skinmasskincareAhh the 90’s… big hair, strong eye shadow and epic shoulder pads. However the 90’s were 30 years ago. Luckily our skincare is from this decade!! Up to date technology so your skincare won’t remind you of how old you really are. SKINmas sk

In [115]:
df

,Unnamed: 0,upload_id,date_time,hashtags,contents,images,location_info,location_href,text
0,0,total_photo2018,2022-04-02T10:04:24.000Z,"['roma', 'castelsantangelo', 'amazing', 'rome'...",total_photo2018📸@barbi.giangiu 🆒💯😍🇮🇹......#rom...,['https://scontent-gmp1-1.cdninstagram.com/v/t...,Castel Sant'angelo - Roma,https://www.instagram.com/explore/locations/10...,total_photo2018📸@barbi.giangiu 🆒💯😍🇮🇹......
1,1,elianndelvalle,2022-04-01T23:52:59.000Z,"['happy', 'fashion', 'vestidoscortos', 'igers'...",elianndelvalleEn esta foto me veo gordita pero...,['https://scontent-gmp1-1.cdninstagram.com/v/t...,Lima - Perú,https://www.instagram.com/explore/locations/10...,elianndelvalleEn esta foto me veo gordita pero...
2,2,pamxtinafighter,2022-04-02T00:04:35.000Z,"['love', 'xtina', 'idol', 'best', 'queen', 'di...",pamxtinafighterCredit : @xtinaquebec 🥰@xtina #...,['https://scontent-gmp1-1.cdninstagram.com/v/t...,NaN,NaN,pamxtinafighterCredit : @xtinaquebec 🥰@xtina
3,3,_cateblanchettfanpage_,2022-04-01T18:32:01.000Z,"['cateblanchett', 'cateblanchettmyqueen', 'cat...",_cateblanchettfanpage_She gave me her usual sm...,['https://scontent-gmp1-1.cdninstagram.com/v/t...,NaN,NaN,_cateblanchettfanpage_She gave me her usual sm...
4,4,anaalokeh,2022-04-02T10:59:38.000Z,"['xiaomiphotography', 'love', 'instagood', 'ph...",anaalokeh💙💜💙#xiaomiphotography .....#love #ins...,['https://scontent-gmp1-1.cdninstagram.com/v/t...,루마니아,https://www.instagram.com/explore/locations/23...,anaalokeh💙💜💙
...,...,...,...,...,...,...,...,...,...
363,363,lucidgrafix10,2022-04-02T10:34:23.000Z,"['design', 'graphicdesign', 'creative', 'aweso...",lucidgrafix10🔥 Logo Designed by @lucidgrafix10...,[],베닌시티,https://www.instagram.com/explore/locations/41...,lucidgrafix10🔥 Logo Designed by @lucidgrafix10...
364,364,lasirgarestaurant,2022-04-02T10:34:27.000Z,"['food', 'foodporn', 'yum', 'instafood', 'me',...",lasirgarestaurantEnsalada a base de mesclum fr...,['https://scontent-gmp1-1.cdninstagram.com/v/t...,NaN,NaN,lasirgarestaurantEnsalada a base de mesclum fr...
365,365,thelifelogics,2022-04-02T10:34:33.000Z,"['Quotes', 'Quote', 'Quoteoftheday', 'Word', '...",thelifelogics💯13시간,['https://scontent-gmp1-1.cdninstagram.com/v/t...,NaN,NaN,thelifelogics💯13시
366,366,lilly_charlottelady,2022-04-02T10:34:25.000Z,"['amyadams', 'redhead', 'redhair', 'motivation...",lilly_charlotteladyHope you’re enjoying your w...,['https://scontent-gmp1-1.cdninstagram.com/v/t...,NaN,NaN,lilly_charlotteladyHope you’re enjoying your w...


In [116]:
for i in range(len(df)):
  post_text= ''
  description = df.iloc[i][-1]
  idx = description.find('@')
  description = description[:idx]
  print(description)
  df['text'][i] = description

total_photo2018📸
elianndelvalleEn esta foto me veo gordita pero aja, a mi me gusto y ya.🤭..Que hacen ?❤11
pamxtinafighterCredit : 
_cateblanchettfanpage_She gave me her usual smile, which stopped my breath and heart.  I couldn't imagine a more splendid angel.❤️❤
anaalokeh💙💜
akrimna_fishcente
robertomullanuLight Swords.........
pandora.carmainSabtu malam minggu enaknya wisata kuliner aja 
piero_hb☔️🌧•••••••
chintu_official5
cricket4fun
mindfullness00
chia_l
chessturFever Dream Pt. 5 : Love Differently...
andreasschnitzlhuberSpringtime .....
insta_klub_4
vishal77975
kotaflorist❤ 👰
sarahsaleh91🦊2
jadorbucProve outfit per il 
daniele.zinna🔥🐲1
fairyheartaqforyou
javad_.golzar
noeeangeliist🍁r
alloccabimboBOW SANDALS WHITE🥛Vi aspettiamo in via Madonna dell'Arco 151,Sant'Anastasia (NA)
jewelplanethydAll New Diamond Finish Jhumka Collection At lowest price Cash On Delivery Available allover India With Best Quality products
reena_reenarai🍑🍑🍑Follw me for more fu
kh__waja*GLitCh* piCtUre.....📸
met

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


melania_giordanoPalermo arriviamo!🔴
iamarbaz_01In the field... ❤
andre_fakCOTOLETTA pt.2||
piccolobucorom
hart.ash1
ludovicafranceschini🖤 ••
hoteljamesjoycetrieste•🇮🇹
lilithstarmusic✨🕊Do you believe in your own magic? Do you see it? 💫..
nofahih56
linnda679
bellissimachaos🏆 Category: Vibe Hol' luuup... my vibe be vibin' 
digitalmusliminSunset Tasik Cempaka Bangi. SelangorSmartphone nova 3
jay_shah_004In love with sunsets 🌅…
the_blevinator123Day 27 of trying to get ninja to follow me.
babycutie.pieReposted from 
vishalpppp
tista_s_kitchenPizza style potato snacks..
emanusicily•🕌🌙•
jdaesthetics_mcrMy lovely lady came back for her two week top up today!💗Before and after’s 😍 💉
facota266
somiaypunt
fcs.anagha
tacej4524
mircocataneseMadeleine 🍬.......
hyderabad_buddies2.0I think  baji ko ____ hogya 🤣🤣🤣🤣Tag Ur friends 🤣😂👇👇
chrissythriftyfindsWell I have been back on plan for nearly 2 weeks and have lost 9 pounds and I’m feeling
nobinbasakLovely Azalea  Flowers 🌸.......
vera_burian.***.... ohne

In [117]:
df

,Unnamed: 0,upload_id,date_time,hashtags,contents,images,location_info,location_href,text
0,0,total_photo2018,2022-04-02T10:04:24.000Z,"['roma', 'castelsantangelo', 'amazing', 'rome'...",total_photo2018📸@barbi.giangiu 🆒💯😍🇮🇹......#rom...,['https://scontent-gmp1-1.cdninstagram.com/v/t...,Castel Sant'angelo - Roma,https://www.instagram.com/explore/locations/10...,total_photo2018📸
1,1,elianndelvalle,2022-04-01T23:52:59.000Z,"['happy', 'fashion', 'vestidoscortos', 'igers'...",elianndelvalleEn esta foto me veo gordita pero...,['https://scontent-gmp1-1.cdninstagram.com/v/t...,Lima - Perú,https://www.instagram.com/explore/locations/10...,elianndelvalleEn esta foto me veo gordita pero...
2,2,pamxtinafighter,2022-04-02T00:04:35.000Z,"['love', 'xtina', 'idol', 'best', 'queen', 'di...",pamxtinafighterCredit : @xtinaquebec 🥰@xtina #...,['https://scontent-gmp1-1.cdninstagram.com/v/t...,NaN,NaN,pamxtinafighterCredit :
3,3,_cateblanchettfanpage_,2022-04-01T18:32:01.000Z,"['cateblanchett', 'cateblanchettmyqueen', 'cat...",_cateblanchettfanpage_She gave me her usual sm...,['https://scontent-gmp1-1.cdninstagram.com/v/t...,NaN,NaN,_cateblanchettfanpage_She gave me her usual sm...
4,4,anaalokeh,2022-04-02T10:59:38.000Z,"['xiaomiphotography', 'love', 'instagood', 'ph...",anaalokeh💙💜💙#xiaomiphotography .....#love #ins...,['https://scontent-gmp1-1.cdninstagram.com/v/t...,루마니아,https://www.instagram.com/explore/locations/23...,anaalokeh💙💜
...,...,...,...,...,...,...,...,...,...
363,363,lucidgrafix10,2022-04-02T10:34:23.000Z,"['design', 'graphicdesign', 'creative', 'aweso...",lucidgrafix10🔥 Logo Designed by @lucidgrafix10...,[],베닌시티,https://www.instagram.com/explore/locations/41...,lucidgrafix10🔥 Logo Designed by
364,364,lasirgarestaurant,2022-04-02T10:34:27.000Z,"['food', 'foodporn', 'yum', 'instafood', 'me',...",lasirgarestaurantEnsalada a base de mesclum fr...,['https://scontent-gmp1-1.cdninstagram.com/v/t...,NaN,NaN,lasirgarestaurantEnsalada a base de mesclum fr...
365,365,thelifelogics,2022-04-02T10:34:33.000Z,"['Quotes', 'Quote', 'Quoteoftheday', 'Word', '...",thelifelogics💯13시간,['https://scontent-gmp1-1.cdninstagram.com/v/t...,NaN,NaN,thelifelogics💯13
366,366,lilly_charlottelady,2022-04-02T10:34:25.000Z,"['amyadams', 'redhead', 'redhair', 'motivation...",lilly_charlotteladyHope you’re enjoying your w...,['https://scontent-gmp1-1.cdninstagram.com/v/t...,NaN,NaN,lilly_charlotteladyHope you’re enjoying your w...


In [123]:
import re
df.iloc[0][1]

'total_photo2018'

In [133]:
for i in range(len(df)):
  description = df.iloc[i][-1]
  id = df.iloc[i][1]
  description = description.replace(id, '')
  pattern = '[^\w\ws]'
  description = re.sub(pattern=pattern, repl=' ', string=description)
  print(description)
  df['text'][i] = description

 
En esta foto me veo gordita pero aja  a mi me gusto y ya    Que hacen   11
Credit   
She gave me her usual smile  which stopped my breath and heart   I couldn t imagine a more splendid angel    
  
akrimna_fishcente
Light Swords         
pandora carmainSabtu malam minggu enaknya wisata kuliner aja 
          
chintu_official5

mindfullness00
chia_l
Fever Dream Pt  5   Love Differently   
Springtime      
insta_klub_4
vishal77975
   
 2
Prove outfit per il 
daniele zinna  1
foryou
javad_ golzar
st r
BOW SANDALS WHITE Vi aspettiamo in via Madonna dell Arco 151 Sant Anastasia  NA 
All New Diamond Finish Jhumka Collection At lowest price Cash On Delivery Available allover India With Best Quality products
   Follw me for more fu
 GLitCh  piCtUre      
meticore_weightloo
     
ahmet_hacifettahogl
_deepthi_090
   
mtwords inReason  Do follow 
hiroya990
lovish10202
Among the most common sayings are  Ramadan Mubarak  and  Ramadan Kareem     These are Arabic sayings that translate to  blessed 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Some landscape views of the MacLeHoseTrailStage4  The Saturday Hike  Hike More  Worry Less    Google Pixel6Pr
   A brand is not one that imitates no one  but one that no one can imitate Andy Stalma
  voglio farmi scivolare il mondo addossoe non scivolare sempre io   
lovish164
pankaj _mayeka
I m an original and that s perfection in itself          
 
nik_ki657
     
lovish695
        
wicked mood
 li stavate aspettando ed ecco qua    Anche gli Easter cookies sono prontiiiiiiii     
  
Spring 
Gymnocalycium hybrid
_mom vibesBack to Brown      
               
 Street Life   BCLM Collection 5 The fifth shot in a collection of images taken at the Black Country Museum  50  of profits made on these prints will support education and conservation projects 
Buraanbur is a cultural dance or vocal incantation performed by Somali women usually at festive gatherings such as weddings it is typically accompanied by a drum beat   East African  Cushitic cultur
Runway  
      
amazing places at worldSu

In [135]:
df['text']

0                                                       
1      En esta foto me veo gordita pero aja  a mi me ...
2                                              Credit   
3      She gave me her usual smile  which stopped my ...
4                                                       
                             ...                        
363                                    Logo Designed by 
364    Ensalada a base de mesclum fresas pina Granada...
365                                                   13
366    Hope you re enjoying your weekend  Good mornin...
367                                     sj surbhi28Summe
Name: text, Length: 368, dtype: object

In [ ]:
pattern = '(http|ftp|https)://(?:[-\w.]|(?:\da-fA-F]{2}))+'  # url 제거
post_text = re.sub(pattern=pattern, repl=' ', string=post_text)
pattern = '[^\w\s]'  # 특수기호 제거
post_text = re.sub(pattern=pattern, repl=' ', string=post_text)
post_text = remove_space(post_text)  # 공백 제거
if not post_text or post_text == ' ' or not post_hashtags: continue
if post_text[0] == ' ': post_text = post_text[1:]